<a href="https://colab.research.google.com/github/kkorolev1/deep-learning-for-drug-design/blob/master/simple-egnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple Impementation of E(n) Equivariant Graph Neural Networks

Original paper https://arxiv.org/pdf/2102.09844.pdf by Victor Garcia Satorras, Emiel Hoogeboom, Max Welling

In [15]:
import time
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

# Load QM9 Dataset

In [5]:
!git clone https://github.com/senya-ashukha/simple-equivariant-gnn.git
%cd simple-equivariant-gnn

Cloning into 'simple-equivariant-gnn'...
remote: Enumerating objects: 87, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 87 (delta 37), reused 31 (delta 5), pack-reused 0
Unpacking objects: 100% (87/87), done.
/content/simple-equivariant-gnn


In [6]:
# QM9 is a dataset for Molecular Property Predictions http://quantum-machine.org/datasets/
# We will predict Highest occupied molecular orbital energy 
# https://en.wikipedia.org/wiki/HOMO_and_LUMO
# We use data loaders from the official repo

from qm9.data_utils import get_data, BatchGraph
train_loader, val_loader, test_loader, charge_scale = get_data(num_workers=1)

dict_keys([0, 1, 6, 7, 8, 9])
dict_keys([0, 1, 6, 7, 8, 9])
dict_keys([0, 1, 6, 7, 8, 9])


# Graph Representation

In [7]:
batch = BatchGraph(iter(train_loader).next(), False, charge_scale)
batch

In the batch: num_graphs 96 num_nodes 1744
> .h 		 a tensor of nodes representations 		shape 1744 x 15
> .x 		 a tensor of nodes positions  			shape 1744 x 3
> .edges 	 a tensor of edges, a fully connected graph 	shape 30764 x 2
> .batch  	 a tensor of graph_ids for each node 		tensor([ 0,  0,  0,  ..., 95, 95, 95])

# Define Equivariant Graph Convs  & GNN

In [8]:
def index_sum(agg_size, source, idx, cuda):
    """
        source is N x hid_dim [float]
        idx    is N           [int]
        
        Sums the rows source[.] with the same idx[.];
    """
    tmp = torch.zeros((agg_size, source.shape[1]))
    tmp = tmp.cuda() if cuda else tmp
    res = torch.index_add(tmp, 0, idx, source)
    return res

In [34]:
class ConvEGNN(nn.Module):
    def __init__(self, in_dim, hid_dim, cuda=True):
        super().__init__()
        self.hid_dim=hid_dim
        self.cuda = cuda
        
        #### **** YOUR CODE HERE **** #####
        # computes messages based on hidden representations -> [0, 1]
        self.f_e = nn.Sequential(
            nn.Linear(in_dim * 2 + 1, hid_dim), nn.SiLU(),
            nn.Linear(hid_dim, hid_dim), nn.SiLU()
        )
        # Tips: See eq. 3 from the paper.
        # Also use nn.Sequential
        
        #### **** YOUR CODE HERE **** #####
        # preducts "soft" edges based on messages 
        self.f_inf = nn.Sequential(
            nn.Linear(hid_dim, 1), nn.Sigmoid()
        )
        # Tips: See eq. 8 from the paper.
        
        #### **** YOUR CODE HERE **** #####
        # updates hidden representations -> [0, 1]
        self.f_h = nn.Sequential(
            nn.Linear(in_dim + hid_dim, hid_dim), nn.SiLU(),
            nn.Linear(hid_dim, hid_dim), nn.SiLU()
        )
        # Tips: See eq. 6 from the paper.
    
    def forward(self, b):
        #### **** YOUR CODE HERE **** #####
        # compute distances for all edges
        e_st, e_end = b.edges[:,0], b.edges[:,1]
        dists = torch.norm(b.x[e_st] - b.x[e_end], dim=1).reshape(-1, 1)
        # dist should be a 2d torch.Tensor of shape (#number_of_edges_in_the_batch, 1)
        
        #### **** YOUR CODE HERE **** #####
        # compute messages
        tmp = torch.hstack([b.h[e_st], b.h[e_end], dists])
        m_ij = self.f_e(tmp)
        # m_ij should be a 2d torch.Tensor of shape (#number_of_edges_in_the_batch, hid_dim)
        
        # predict edges
        e_ij = self.f_inf(m_ij)
        
        # average e_ij-weighted messages  
        # m_i is num_nodes x hid_dim
        m_i = index_sum(b.h.shape[0], e_ij*m_ij, b.edges[:,0], self.cuda)
        
        #### **** YOUR CODE HERE **** #####
        # update hidden representations
        b.h += self.f_h(torch.hstack([b.h, m_i]))
        # b.h should be a 2d torch.Tensor of shape (#number_of_nodes_in_the_batch, hid_dim)
        # see appendix C. Implementatation details

        return b

In [35]:
class NetEGNN(nn.Module):
    def __init__(self, in_dim=15, hid_dim=128, out_dim=1, n_layers=7, cuda=True):
        super().__init__()
        self.hid_dim=hid_dim
        
        self.emb = nn.Linear(in_dim, hid_dim) 

        #### **** YOUR CODE HERE **** #####
        # Make gnn of n_layers
        self.gnn = nn.Sequential(*[ConvEGNN(hid_dim, hid_dim, cuda) for _ in range(n_layers)])
        # use ConvEGNN layer
        
        self.pre_mlp = nn.Sequential(
            nn.Linear(hid_dim, hid_dim), nn.SiLU(),
            nn.Linear(hid_dim, hid_dim))
        
        self.post_mlp = nn.Sequential(
            nn.Dropout(0.4),
            nn.Linear(hid_dim, hid_dim), nn.SiLU(),
            nn.Linear(hid_dim, out_dim))

        if cuda: self.cuda()
        self.cuda = cuda
    
    def forward(self, b):
        b.h = self.emb(b.h)
        
        b = self.gnn(b)
        h_nodes = self.pre_mlp(b.h)
        
        # h_graph is num_graphs x hid_dim
        h_graph = index_sum(b.nG, h_nodes, b.batch, self.cuda) 
        
        out = self.post_mlp(h_graph)
        return out

In [36]:
epochs = 1000
cuda = True

model = NetEGNN(n_layers=7, cuda=cuda)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-16)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs, verbose=False)
loss_fn = nn.L1Loss()

# Training

In [ ]:
print('> start training')

tr_ys = train_loader.dataset.data['homo'] 
me, mad = torch.mean(tr_ys), torch.mean(torch.abs(tr_ys - torch.mean(tr_ys)))

if cuda:
    me = me.cuda()
    mad = mad.cuda()

train_loss = []
val_loss = []
test_loss = []

for epoch in range(epochs):
    print('> epoch %s:' % str(epoch).zfill(3), end=' ', flush=True) 
    start = time.time()

    batch_train_loss = []
    batch_val_loss = []
    batch_test_loss = []

    model.train()
    for batch in train_loader:
        batch = BatchGraph(batch, cuda, charge_scale)
        
        out = model(batch).reshape(-1)
        #### **** YOUR CODE HERE **** #####
        # compute l1-loss 
        
        loss =  loss_fn(out, (batch.y - me) / mad)
        # use (batch.y-me)/mad as the true value

        loss.backward()
        #### **** YOUR CODE HERE **** #####
        # make step of opimizer 
        # google it
        optimizer.step()
        optimizer.zero_grad()
        
        with torch.no_grad():
            loss =  F.l1_loss(out*mad+me, batch.y)

        batch_train_loss += [float(loss.data.cpu().numpy())]  
        
    train_loss += [np.mean(batch_train_loss)/0.001]
    
    print('train %.3f' % train_loss[-1], end=' ', flush=True)
    
    with torch.no_grad():
        model.eval()
        for batch in val_loader:
            batch = BatchGraph(batch, cuda, charge_scale)
            out = model(batch).reshape(-1)
            loss = F.l1_loss(out*mad+me, batch.y).data.cpu().numpy()
            batch_val_loss += [np.mean(loss)]
            
        val_loss += [np.mean(batch_val_loss)/0.001]
        
        print('val %.3f' % val_loss[-1], end=' ', flush=True)
        
        for batch in test_loader:
            batch = BatchGraph(batch, cuda, charge_scale)
            out = model(batch).reshape(-1)
            loss = F.l1_loss(out*mad+me, batch.y).data.cpu().numpy()
            batch_test_loss += [np.mean(loss)]

        test_loss += [np.mean(batch_test_loss)/0.001]
        
    end = time.time()

    print('test %.3f (%.1f sec)' % (test_loss[-1], end-start), flush=True)
    lr_scheduler.step()

> start training
> epoch 000: train 385.537 val 310.069 test 309.351 (89.0 sec)
> epoch 001: train 273.141 val 227.667 test 224.517 (88.8 sec)
> epoch 002: train 215.969 val 188.897 test 187.103 (88.1 sec)
> epoch 003: train 188.004 val 168.221 test 166.742 (87.5 sec)
> epoch 004: train 170.863 val 153.333 test 152.677 (87.9 sec)
> epoch 005: train 153.744 val 138.983 test 138.773 (87.9 sec)
> epoch 006: train 142.101 val 136.564 test 136.870 (87.4 sec)
> epoch 007: train 132.332 val 118.766 test 118.718 (87.5 sec)
> epoch 008: train 124.101 val 116.802 test 116.612 (87.1 sec)
> epoch 009: train 117.104 val 109.574 test 109.298 (87.7 sec)
> epoch 010: train 111.843 val 97.765 test 97.931 (88.1 sec)
> epoch 011: train 106.154 val 93.443 test 93.085 (88.0 sec)
> epoch 012: train 100.970 val 99.415 test 98.931 (88.0 sec)
> epoch 013: train 98.066 val 98.721 test 99.270 (87.3 sec)
> epoch 014: train 94.752 val 87.757 test 87.408 (87.8 sec)
> epoch 015: train 90.951 val 84.208 test 84.663 (

In [ ]:
# This is good results 
# Lower is better
# val 30.157 test 30.886